In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from baa import PerplexityBenchmark, get_llm_memory_usage, device_map
from datasets import load_dataset

In [ ]:
model_name = "HuggingFaceTB/SmolLM-1.7B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map=device_map)
quantized_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
)
franken_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(model)

In [ ]:
print(f"Full Model Memory Usage: {get_llm_memory_usage(model.model) / 1024**2:.2f} MB")
print(f"Embedding Memory Usage: {get_llm_memory_usage(model.model.embed_tokens) / 1024**2:.2f} MB")
print(f"Attention Heads Memory Usage: {get_llm_memory_usage(model.model.layers[:]) / 1024**2:.2f} MB")
inner_self_attn_mapping = {
    "self_attn": model.model.layers[0].self_attn,
    "mlp": model.model.layers[0].mlp,
}

for key, value in inner_self_attn_mapping.items():
    print(f"{key} Memory Usage: {get_llm_memory_usage(value) / 1024**2:.2f} MB")

print(f"Model Head Memory Usage: {get_llm_memory_usage(model.lm_head) / 1024**2:.2f} MB")